# Exploratory Data Analysis and Cleaning

## Author: 
## Date: OCT 10, 2023

### Table of contents

### Introduction

This notebook does the data cleaning for the accepted loans. Due to the dataset size, a sample CSV file is used to increase performance.

### Data Dictionary

- how much data is lost per column due to cleaning
- no data truncation subprocess, load from drive
- explain why data is missing etc
- 

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from pathlib import Path

#### Load in the data

Place data within the Data

In [10]:
#pathlib is used to ensure compatibility across operating systems
try:
    data_destination = Path('../Data/Lending_club/sample_accepted_2007_to_2018Q4.csv')
    sample_accepted_df = pd.read_csv(data_destination)
except FileNotFoundError as e:
    print(e.args[1])
    print('Check file location')

C:\Users\pags\AppData\Local\Temp\ipykernel_12492\217665415.py:4: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  sample_accepted_df = pd.read_csv(data_destination)


In [16]:
sample_accepted_df.head(5)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


The are too many columns to analyze at once

#### Seperate Columns by datatype

In [18]:
numeric_sample_accepted_df = sample_accepted_df.select_dtypes(include=['number'])
numeric_sample_accepted_df

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,...,deferral_term,hardship_amount,hardship_length,hardship_dpd,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,13.99,123.03,55000.0,5.91,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,11.99,820.28,65000.0,16.06,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,10.78,432.66,63000.0,10.78,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.0,35000.0,35000.0,14.85,829.90,110000.0,17.06,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.0,10400.0,10400.0,22.45,289.91,104433.0,25.37,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,64360310,NaN,12600.0,12600.0,12600.0,11.99,280.22,104000.0,15.06,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19996,67475682,NaN,29825.0,29825.0,29825.0,13.67,688.89,140000.0,11.93,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19997,67245737,NaN,6000.0,6000.0,6000.0,10.64,195.42,88000.0,24.64,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19998,67265833,NaN,19000.0,19000.0,19000.0,10.64,618.81,75000.0,12.59,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
empty_columns = numeric_sample_accepted_df.columns[numeric_sample_accepted_df.isna().all()]
empty_columns

Index(['member_id', 'revol_bal_joint', 'sec_app_fico_range_low',
       'sec_app_fico_range_high', 'sec_app_earliest_cr_line',
       'sec_app_inq_last_6mths', 'sec_app_mort_acc', 'sec_app_open_acc',
       'sec_app_revol_util', 'sec_app_open_act_il', 'sec_app_num_rev_accts',
       'sec_app_chargeoff_within_12_mths',
       'sec_app_collections_12_mths_ex_med',
       'sec_app_mths_since_last_major_derog'],
      dtype='object')

In [13]:
rows, cols = sample_accepted_df.shape
print('Number of Rows: ',rows)
print('Number of Columns: ', cols)

Number of Rows:  20000
Number of Columns:  151


In [14]:
sample_accepted_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Columns: 151 entries, id to settlement_term
dtypes: float64(114), int64(1), object(36)
memory usage: 23.0+ MB


In [39]:
sample_accepted_df.describe()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,...,deferral_term,hardship_amount,hardship_length,hardship_dpd,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,settlement_amount,settlement_percentage,settlement_term
count,2.000000e+04,0.0,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,2.000000e+04,19999.000000,20000.000000,...,162.0,162.000000,162.0,162.000000,129.000000,162.000000,162.000000,583.000000,583.000000,583.000000
mean,6.765001e+07,NaN,15048.700000,15048.700000,15042.995000,12.251883,437.311208,7.833469e+04,19.415594,0.335500,...,3.0,133.094136,3.0,14.327160,398.565814,10761.745370,181.755370,4989.886895,47.156123,13.996569
std,1.904111e+06,NaN,8735.526302,8735.526302,8730.708785,4.208918,249.614324,6.054772e+04,11.260728,0.887682,...,0.0,110.905181,0.0,10.152765,326.868419,6597.328315,180.698584,3604.108691,5.267301,7.503492
min,3.617740e+05,NaN,1000.000000,1000.000000,1000.000000,5.320000,30.540000,0.000000e+00,0.000000,0.000000,...,3.0,5.280000,3.0,0.000000,33.840000,594.070000,0.060000,250.000000,30.000000,0.000000
25%,6.734711e+07,NaN,8000.000000,8000.000000,8000.000000,9.170000,255.040000,4.800000e+04,12.655000,0.000000,...,3.0,48.737500,3.0,5.250000,146.910000,5861.255000,48.392500,2085.930000,45.000000,8.000000
50%,6.769533e+07,NaN,14000.000000,14000.000000,13962.500000,11.990000,381.425000,6.700000e+04,18.830000,0.000000,...,3.0,102.855000,3.0,16.000000,320.520000,9526.755000,119.425000,4178.730000,45.000000,14.000000
75%,6.824256e+07,NaN,20000.000000,20000.000000,20000.000000,14.480000,581.580000,9.500000e+04,25.640000,0.000000,...,3.0,182.152500,3.0,24.000000,527.370000,15115.545000,266.660000,7174.400000,50.000000,18.000000
max,6.861706e+07,NaN,35000.000000,35000.000000,35000.000000,28.990000,1354.660000,3.964280e+06,999.000000,15.000000,...,3.0,629.700000,3.0,30.000000,1889.100000,28479.590000,780.050000,17500.000000,67.450000,36.000000


In [41]:
sample_accepted_df.isnull().sum()

id                           0
member_id                20000
loan_amnt                    0
funded_amnt                  0
funded_amnt_inv              0
                         ...  
settlement_status        19417
settlement_date          19417
settlement_amount        19417
settlement_percentage    19417
settlement_term          19417
Length: 151, dtype: int64

sam

In [42]:
objectsample_accepted_df.select_dtypes(object)

,term,grade,sub_grade,emp_title,emp_length,home_ownership,verification_status,issue_d,loan_status,pymnt_plan,...,hardship_status,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_loan_status,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date
0,36 months,C,C4,leadman,10+ years,MORTGAGE,Not Verified,Dec-2015,Fully Paid,n,...,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN
1,36 months,C,C1,Engineer,10+ years,MORTGAGE,Not Verified,Dec-2015,Fully Paid,n,...,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN
2,60 months,B,B4,truck driver,10+ years,MORTGAGE,Not Verified,Dec-2015,Fully Paid,n,...,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN
3,60 months,C,C5,Information Systems Officer,10+ years,MORTGAGE,Source Verified,Dec-2015,Current,n,...,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN
4,60 months,F,F1,Contract Specialist,3 years,MORTGAGE,Source Verified,Dec-2015,Fully Paid,n,...,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,60 months,C,C1,Civil Aviation Security Specialist,10+ years,RENT,Not Verified,Dec-2015,Current,n,...,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN
19996,60 months,C,C4,Sales manager,10+ years,OWN,Source Verified,Dec-2015,Charged Off,n,...,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN
19997,36 months,B,B4,Teacher,8 years,RENT,Not Verified,Dec-2015,Fully Paid,n,...,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN
19998,36 months,B,B4,Business Banking Specialist,3 years,MORTGAGE,Source Verified,Dec-2015,Fully Paid,n,...,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN


### Resources used:

- https://stackoverflow.com/questions/3777301/how-to-call-a-shell-script-from-python-code